# Prefix-tuning 实战

## step1 import lib

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## step2 load datasets

In [2]:
ds = Dataset.load_from_disk(r"../dataset/alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[1:4]

{'output': ['4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。',
  '法国的首都是巴黎。'],
 'input': ['输入：4/16', '', ''],
 'instruction': ['解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？', '法国的首都是什么？']}

## step3 data process

In [4]:
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer

BloomTokenizerFast(name_or_path='D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: "+ example["instruction"], example['input']]).strip() + "\n\nAssistant: ")
    response = tokenizer(example['output'] + tokenizer.eos_token)

    # 拼接
    input_ids = instruction['input_ids'] + response['input_ids']
    attention_mask = instruction['attention_mask'] + response['attention_mask']
    labels = [-100] * len(instruction['input_ids']) + response['input_ids']

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]['input_ids'])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [8]:
# decode看下 `labels`
tokenizer.decode(list(filter(lambda x : x != -100, tokenized_ds[1]['labels'])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## step4 create model

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
for param in model.parameters():
    print("param: ", param,  "\nnumbers: ", param.numel(), "\n", "="*100)

In [10]:
sum(param.numel() for param in model.parameters())

345768960

### 显存占用

model size: 0.35B

model: 0.35G * 4 ~= 1.4G

gradient: 0.35G * 4 ~= 1.4G

optimizer: 0.35G * 4 * 2 ~= 2.8G

sum: 5.6G

## Prefix-tuning 部分

跟 `p-tuning` 很像，但是 `p-tuning` 是在embedding层进行拼接； `prefix-tuning` 是在整个transformers blocks 进行拼接然后进行学习的。

> 详细可以看下 `prft.get_peft_model` 里面的 `get_prompt` 方法是怎么处理的 (`prefix-tuning` 比较特殊，有特别处理的一环)

### PEFT Step1 配置文件

这里的 `prefix_projection=True` 跟之前的 `p-tuning` 类似，都是重参数化的方式（可以看源码，也是加了2层MLP层）

In [11]:
from peft import PrefixTuningConfig, get_peft_model, TaskType

config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10,
    prefix_projection=True
)

config

PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=True)

### PEFT Step2 创建模型

In [12]:
model = get_peft_model(model, config)

In [13]:
model.prompt_encoder

ModuleDict(
  (default): PrefixEncoder(
    (embedding): Embedding(10, 1024)
    (transform): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): Tanh()
      (2): Linear(in_features=1024, out_features=49152, bias=True)
    )
  )
)

In [14]:
model.print_trainable_parameters()

trainable params: 51,440,640 || all params: 397,209,600 || trainable%: 12.9505


## step5 config train parameters

In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=100,
    num_train_epochs=2
)

## step6 create trainer

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## step7 model train

In [ ]:
trainer.train()

## step8 model inference

In [20]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 1. 在考试前，准备好各种材料，以便提前做好准备。例如：需要准备一份试卷，一份答案集，一本参考书目等等。这些材料可以帮助你了解考试要求的内容和要求的方式，从而更好的给考试准备提前做好准备。'

In [18]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [19]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
text = pipe(ipt, max_length=256, do_sample=True, )
text

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 首先，考生首先对考试内容及试题应考要求进行了解，并积极备考，以取得较好的考试成绩。比如，根据高考科目需要，考生应该会优先考虑选择题，因为其考查的范围比较广泛，涉及的知识也相对专业。另外，准备考试时，考生还必须关注考试形式，选择题往往具有考查的内容、难点等较强的综合性，考生可以根据情况选择最适合自己的。'}]

In [ ]:
ans = text[0]['generated_text']
print(ans)